In [1]:
cd ~/src/Classes/Ansible

/Users/rick446/src/Classes/Ansible


# Playbook error handling and debugging

What to do when it doesn't "just work"

## Before trying it, you should test with a dry run

In [3]:
%%bash
ansible-playbook -i inventory --check playbooks/setup-user.yaml -e username=rick


PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [db]
ok: [web]
ok: [arborian-02.class.arborian.com]
ok: [arborian-03.class.arborian.com]
ok: [arborian-05.class.arborian.com]
ok: [arborian-06.class.arborian.com]
ok: [arborian-04.class.arborian.com]
ok: [arborian-01.class.arborian.com]

TASK [Let the sudo group use sudo without a password] **************************
ok: [db]
ok: [web]
ok: [arborian-02.class.arborian.com]
ok: [arborian-03.class.arborian.com]
ok: [arborian-01.class.arborian.com]
ok: [arborian-05.class.arborian.com]
ok: [arborian-04.class.arborian.com]
ok: [arborian-06.class.arborian.com]

TASK [create user] *************************************************************
ok: [db]
ok: [web]
ok: [arborian-02.class.arborian.com]
ok: [arborian-01.class.arborian.com]
ok: [arborian-03.class.arborian.com]
ok: [arborian-05.class.arborian.com]
ok: [arborian-04.class.

## First Attempt: Rerun in verbose mode

Sometimes you just need to see the actual commands that Ansible is running:

In [4]:
%%bash
ansible-playbook -vv playbooks/setup-user.yaml -e username=rick

ansible-playbook 2.4.2.0
  config file = /Users/rick446/src/Classes/Ansible/ansible.cfg
  configured module search path = ['/Users/rick446/.ansible/plugins/modules', '/usr/share/ansible/plugins/modules']
  ansible python module location = /Users/rick446/.virtualenvs/ansible/lib/python3.6/site-packages/ansible
  executable location = /Users/rick446/.virtualenvs/ansible/bin/ansible-playbook
  python version = 3.6.0 (default, Dec 30 2016, 13:36:14) [GCC 4.2.1 Compatible Apple LLVM 8.0.0 (clang-800.0.42.1)]
Using /Users/rick446/src/Classes/Ansible/ansible.cfg as config file

PLAYBOOK: setup-user.yaml ******************************************************
1 plays in playbooks/setup-user.yaml

PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [db]
ok: [web]
META: ran handlers

TASK [Let the sudo group use sudo without a password] **************************
task path: /Users/ri

## Second attempt: Playbook debugger

We can also add a `strategy: debug` to a play which will drop us into an interactive debugger if a task fails:


In [6]:
cat playbooks/debug-demo.yaml

- hosts: localhost
  gather_facts: false
  strategy: debug
  tasks:
  - name: Run my simple module
    simple_module:
      # name: fail me Missing name!
      new: true


## Third attempt: spam `debug` module statements

Sometimes when you need visibility, you can just have Ansible print out messages as it executes your playbook:

In [5]:
cat playbooks/vault-demo2.yaml

- hosts: localhost
  gather_facts: false
  vars:
    encrypted_value: !vault |
          $ANSIBLE_VAULT;1.1;AES256
          31393366356333313566346465333839643761653065623062363230386662376262313663363539
          6635366137386261353731323065313832303933313036630a613231363333643363373762373937
          31623565316532323132346663343834306562386137313032313537613465336631656630616338
          6639306530666464610a313061343966646262393530323931636639353163396637306334616133
          36653830323234323330376531363537653965306137623033636136613534356363
  tasks:
    - name: Show the encrypted value
      debug: var=encrypted_value


In [6]:
%%bash
ansible web -m debug -a 'msg="This is running on {{ansible_host}}"'

web | SUCCESS => {
    "changed": false,
    "msg": "This is running on 127.0.0.1"
}


## Final attempt: single-stepping

Ansible supports single-stepping through a playbook, allowing you to manually examine the hosts as the playbook runs.

`ansible-playbook --step setup-stack-cached.yaml`

# Playbook Error Handling

How to handle cases where your playbook is fine, but the command running remotely had an error?

### Ignoring failed commands

You can always include `ignore_errors: true` on any task and Ansible just won't care

### Unreachable hosts

If a host is unreachable for *one* task, it is removed from *all* tasks and plays for that playbook. To re-enable it (and any other hosts that are being suppressed), you can include

```
meta: clear_host_errors
```

to re-activate all failed hosts.

### Handler behavior in errors

If a task fails in a play, any handlers which were notified will, by default, *not* run. To change this behavior, you can:

- Use the `--force-handlers` command-line option
- Include `force_handlers: true` in a play
- Include `force_handlers = True` in `ansible.cfg`

### What does it really mean to "fail", anyway?

Normally, Ansible uses the result code of a command to define failure. Sometimes you want something else (like checking for some string in a command output). You can configure this using the `failed_when` flag on a task. Some examples from http://docs.ansible.com/ansible/latest/playbooks_error_handling.html:
```
- name: Fail task when the command error output prints FAILED
  command: /usr/bin/example-command -x -y -z
  register: command_result
  failed_when: "'FAILED' in command_result.stderr"
- name: Fail task when both files are identical
  raw: diff foo/file1 bar/file2
  register: diff_cmd
  failed_when: diff_cmd.rc == 0 or diff_cmd.rc >= 2
```

### And what about "changed"?

Ansible also similarly supports a `changed_when` key in tasks to determine when a task has changed something (and thus its notifiers should run). For instance, the following task will never report 'changed':

```
  - shell: wall 'beep'
    changed_when: False
```

### Aborting the whole play

Normally, Ansible tries to complete a play for as many hosts as possible. You may instead have Ansible mark *all* hosts failed by including the `any_errors_fatal: true` flag in a play.


# Lab: Modify error handling

Using the custom module that we created before, make a playbook that fails a task when running on 'all' hosts